# 搜索

主要分为DFS和BFS，部分题目可以使用并查集

相关题目列表：
- [岛屿数量](https://leetcode-cn.com/problems/number-of-islands/)
- [T秒后青蛙的位置](https://leetcode-cn.com/problems/frog-position-after-t-seconds/)


## Id200-岛屿数量

[岛屿数量](https://leetcode-cn.com/problems/number-of-islands/)，利用DFS、BFS和并查集完成此题。很基础的一道模板题，需要考虑诸如：上下左右四个方向移动、边界判断、节点是否被访问过和题目给定的一些判断条件。并查集的方法思路巧妙，但是运行效率一般。


In [6]:
### 方法1: DFS
class Solution:
    def numIslands(self, grid) -> int:
        def dfs(grid, i, j, m, n, used):
            used[i][j] = True

            steps = [[1, 0], [0, 1], [-1, 0], [0, -1]]
            for dx, dy in steps:
                newx = i + dx
                newy = j + dy
                if 0 <= newx < m and 0 <= newy < n and not used[newx][newy] and grid[newx][newy] == "1":
                    dfs(grid, newx, newy, m, n, used)

        m = len(grid)
        if m == 0:
            return 0
        n = len(grid[0])

        count = 0
        used = [[False for _ in range(n)] for _ in range(m)]

        for i in range(m):
            for j in range(n):
                if grid[i][j] == "1" and (not used[i][j]):
                    dfs(grid, i, j, m, n, used)
                    count += 1

        return count


if __name__ == '__main__':
    s = Solution()
    grid = [["1", "1", "1", "1", "0"], ["1", "1", "0", "1", "0"], ["1", "1", "0", "0", "0"], ["0", "0", "0", "0", "0"]]
    print(s.numIslands(grid))

1


In [7]:
### 方法2: BFS
class Solution:
    def numIslands(self, grid) -> int:
        from queue import Queue

        m = len(grid)
        if m == 0:
            return 0
        n = len(grid[0])

        count = 0
        used = [[False for _ in range(n)] for _ in range(m)]

        for i in range(m):
            for j in range(n):
                if grid[i][j] == "1" and (not used[i][j]):
                    que = Queue()
                    que.put((i, j))
                    count += 1

                    while not que.empty():
                        cur_x, cur_y = que.get()
                        steps = [[1, 0], [0, 1], [-1, 0], [0, -1]]
                        for dx, dy in steps:
                            new_x = cur_x + dx
                            new_y = cur_y + dy
                            if 0 <= new_x < m and 0 <= new_y < n and not used[new_x][new_y] \
                                    and grid[new_x][new_y] == "1":
                                que.put((new_x, new_y))
                                used[new_x][new_y] = True

        return count
    
if __name__ == '__main__':
    s = Solution()
    grid = [["1", "1", "1", "1", "0"], ["1", "1", "0", "1", "0"], ["1", "1", "0", "0", "0"], ["0", "0", "0", "0", "0"]]
    print(s.numIslands(grid))

1


In [8]:
### 方法3: 并查集
'''
并查集demo
'''
class Solution:
    def numIslands(self, grid) -> int:
        # UnionFind Set
        class UnionFind:
            def __init__(self, n):
                self.count = n
                self.parent = [i for i in range(n)]
                self.rank = [1 for _ in range(n)]

            def get_count(self):
                return self.count

            def find(self, p):
                while p != self.parent[p]:
                    self.parent[p] = self.parent[self.parent[p]]
                    p = self.parent[p]
                return p

            def union(self, p, q):
                p_root = self.find(p)
                q_root = self.find(q)
                if p_root == q_root: return

                if self.rank[p_root] > self.rank[q_root]:
                    self.parent[q_root] = p_root
                    self.rank[p_root] += 1
                else:
                    self.parent[p_root] = q_root
                    self.rank[q_root] += 1

                self.count -= 1

        # main
        row = len(grid)
        if row == 0:
            return 0
        col = len(grid[0])

        def get_index(x, y):
            return x * col + y

        # 实际节点编号从 0 ~ row*col-1，虚拟节点编号row * col
        dummy_node = row * col  # 多出来的一个节点，虚拟节点
        uf = UnionFind(dummy_node + 1)  # 多开一个空间，把水域 "0" 都归到这个虚拟节点
        for i in range(row):
            for j in range(col):
                if grid[i][j] == '0':
                    uf.union(get_index(i, j), dummy_node)
                else:
                    for dx, dy in [[1, 0], [0, 1]]:
                        new_x = i + dx
                        new_y = j + dy
                        if new_x < row and new_y < col and grid[new_x][new_y] == '1':
                            uf.union(get_index(i, j), get_index(new_x, new_y))

        return uf.get_count() - 1  # 删除虚拟节点

if __name__ == '__main__':
    s = Solution()
    grid = [["1", "1", "1", "1", "0"], ["1", "1", "0", "1", "0"], ["1", "1", "0", "0", "0"], ["0", "0", "0", "0", "0"]]
    print(s.numIslands(grid))

1


## Id5355-T秒后青蛙的位置

[T秒后青蛙的位置](https://leetcode-cn.com/problems/frog-position-after-t-seconds/)，这道题目算是BFS中比较复杂的：
- 建立树的过程
- 访问过的点不能在访问
- 到达target后的处理

In [9]:
class Solution:
    def frogPosition(self, n: int, edges, t: int, target: int) -> float:
        if n == 1:
            return 1.0
        
        next_nodes = [[] for _ in range(n + 1)]
        for edge in edges:
            next_nodes[edge[0]].append(edge[1])
            next_nodes[edge[1]].append(edge[0])

        from queue import Queue
        que = Queue()
        que.put((1, 0, 1.0))  # id, time, prob
        travel = set([1])
        while not que.empty():
            id, ct, prob = que.get()

            new_next_nodes = []
            for next_node in next_nodes[id]:
                if next_node not in travel:
                    travel.add(next_node)
                    new_next_nodes.append(next_node)
            for next_node in new_next_nodes:
                next_node_prob = prob / len(new_next_nodes)
                if next_node == target:
                    if ct + 1 == t:
                        return next_node_prob
                    elif ct + 1 < t:
                        for nnode in next_nodes[target]:
                            if nnode not in travel:
                                return 0.0
                        return next_node_prob
                    else:
                        return 0.0
                if ct + 1 < t:
                    que.put((next_node, ct + 1, next_node_prob))

        return 0.0


if __name__ == '__main__':
    s = Solution()
    print(s.frogPosition(n=7, edges=[[1, 2], [1, 3], [1, 7], [2, 4], [2, 6], [3, 5]], t=2, target=4))
    print(s.frogPosition(n=3, edges=[[2, 1], [3, 2]], t=1, target=2))

0.16666666666666666
1.0
